In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Import data
ffdaily = pd.read_csv('ff6_daily_factors.csv')
ffmonthly = pd.read_csv('ff6_monthly_factors.csv')
factorlist = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MOM']

### Q1 (a)-(f) for all six factors.

In [3]:
factordaily_res = pd.read_csv('ff6_daily_factors.csv', usecols = [0])

for factor in factorlist:
    factordaily = ffdaily[['date', factor]].copy()
    # Calculate rolling variance
    factordaily['rolling_var_'+factor] = factordaily[factor].rolling(window=22, min_periods=22).var()
    # Calculate factor weight
    targetvol = factordaily['rolling_var_'+factor].mean()
    factordaily['weight_'+factor] = targetvol/factordaily['rolling_var_'+factor]
    # Collapse to Monthly
    factordaily['mdate'] = (factordaily['date']/100).astype(int)
    factordaily = factordaily.drop_duplicates('mdate', keep='last')
    
    factordaily = factordaily[['mdate', 'weight_'+factor, 'rolling_var_'+factor]].copy()
    factordaily = factordaily.rename(columns={'mdate':'date'})
    factordaily_res = pd.merge(factordaily_res, factordaily, on='date', how='right')

# Merge weights
factormonthly = pd.merge(ffmonthly, factordaily_res, on='date', how='left')

# Is volatility persistent?
for factor in factorlist:
    voldf = factormonthly[['date', 'rolling_var_'+factor, factor]].copy()
    voldf['lag_vol'] = voldf['rolling_var_'+factor].shift(1)
    voldf = voldf.dropna()
    modelvol=sm.OLS(voldf['rolling_var_'+factor],
            sm.add_constant(voldf[['lag_vol']])).fit()
    print(modelvol.summary())
    # Does Vol predict factor returns?
    modelvol=sm.OLS(voldf[factor],
            sm.add_constant(voldf[['lag_vol']])).fit()
    print(modelvol.summary())

for factor in factorlist:
    factormonthly['Ret_'+factor] = factormonthly['weight_'+factor].shift(1)*(factormonthly[factor])
    
# Asset pricing test of the CAPM
for factor in factorlist:
    cols = ['Ret_'+factor, factor]
    returndf = factormonthly[cols].copy().dropna()
    model1=sm.OLS(returndf['Ret_'+factor],
            sm.add_constant(returndf[[factor]])).fit()
    print(model1.summary())

# AP Test FF3
for factor in factorlist:
    cols = ['Ret_'+factor, 'Mkt-RF','SMB', 'HML']
    returndf = factormonthly[cols].copy().dropna()
    model2=sm.OLS(returndf['Ret_'+factor],
        sm.add_constant(returndf[['Mkt-RF','SMB', 'HML']])).fit()
    print(model2.summary())

# AP Test FF5 + MOM
for factor in factorlist:
    cols = ['Ret_'+factor, 'Mkt-RF','SMB', 'HML', 'RMW', 'CMA', 'MOM']
    returndf = factormonthly[cols].copy().dropna()
    model3=sm.OLS(returndf['Ret_'+factor],
        sm.add_constant(returndf[['Mkt-RF','SMB', 'HML', 'RMW', 'CMA', 'MOM']])).fit()
    print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:     rolling_var_Mkt-RF   R-squared:                       0.287
Model:                            OLS   Adj. R-squared:                  0.287
Method:                 Least Squares   F-statistic:                     466.4
Date:                Sun, 01 Oct 2023   Prob (F-statistic):           3.39e-87
Time:                        15:09:51   Log-Likelihood:                -2535.5
No. Observations:                1160   AIC:                             5075.
Df Residuals:                    1158   BIC:                             5085.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5418      0.070      7.790      0.0

                            OLS Regression Results                            
Dep. Variable:                Ret_HML   R-squared:                       0.331
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     190.5
Date:                Sun, 01 Oct 2023   Prob (F-statistic):          2.27e-100
Time:                        15:09:51   Log-Likelihood:                -4116.0
No. Observations:                1160   AIC:                             8240.
Df Residuals:                    1156   BIC:                             8260.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5757      0.250      2.303      0.0